## DDQN in Hockey Enviroment - Evaluation

Evaluate a trained DDQN against various environments

Base DQN implementation adapted from HW7

In [2]:
import os
import sys
from importlib import reload

import numpy as np
from tqdm.notebook import tqdm

# Adding the parent directory to the path to enable importing
root_dir = os.path.dirname(os.path.abspath("../"))
if root_dir not in sys.path:
    sys.path.append(root_dir)

import DDQN.DDQN as ddqn
from DDQN.DDQN import DDQNAgent
from DDQN.evaluation import compare_agents, display_stats
import hockey.hockey_env as h_env

reload(h_env)
reload(ddqn)

/home/kivanc/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment Hockey-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/kivanc/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment Hockey-One-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


<module 'DDQN.DDQN' from '/home/kivanc/ders/RL/project/RL-Hockey/DDQN/DDQN.py'>

In [3]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

## Environment & Agent Initialization

In [4]:
env = h_env.HockeyEnv(mode=h_env.Mode.NORMAL)
env.reset()

(array([-3.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  3.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.75742769, -0.70681381,  0.        ,
         0.        ,  0.        ,  0.        ]),
 {'winner': 0,
  'reward_closeness_to_puck': 0.0,
  'reward_touch_puck': 0.0,
  'reward_puck_direction': 0.0})

In [ ]:
STATE_PATH = "../models/hockey_mixed_dev/"

agent_player = DDQNAgent(
    env.observation_space,
    env.discrete_action_space,
    hidden_sizes=[256, 256],
)
agent_player.load_state(STATE_PATH)

agent_opp_weak = h_env.BasicOpponent(weak=True)
agent_opp_strong = h_env.BasicOpponent(weak=False)

## Evaluation

### Winning Rates Against Opponents & Match Statistics

In [6]:
weak_opp_stats = compare_agents(
    agent_player, agent_opp_weak, env, num_matches=1000, tqdm=tqdm
)
strong_opp_stats = compare_agents(
    agent_player, agent_opp_strong, env, num_matches=1000, tqdm=tqdm
)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
print("Weak Opponent Stats:")
display_stats(weak_opp_stats)

print("Strong Opponent Stats:")
display_stats(strong_opp_stats)

Weak Opponent Stats:
Player Observation Mean:
  x pos player one: -2.0549537832733447
  y pos player one: -0.9174791878218195
  angle player one: 0.02201530391115669
  x vel player one: 0.35400822789872183
  y vel player one: -0.5575216505351985
  angular vel player one: 0.006990826031689874
  x player two: 2.5092787907211473
  y player two: 0.014472255215503474
  angle player two: -0.001694797866540479
  y vel player two: -0.019542543775540197
  y vel player two: -0.01968256426456817
  angular vel player two: -0.004213378888354722
  x pos puck: -0.22512376772053289
  y pos puck: -0.024328366624081478
  x vel puck: -1.006091140812737
  y vel puck: 0.0668359821028479
  left player puck keep time: 0.7734816310524076
  right player puck keep time: 0.7753506495673723

Relative Std. Change in Agent Observations:
  x pos player one: -0.057341693770048975
  y pos player one: 0.5533949891603777
  angle player one: 0.7598650992815751
  x vel player one: -0.02615624890885328
  y vel player one: 

### Rendered Demonstration Against Strong Opponent

In [8]:
obs_buffer = []
reward_buffer = []
obs, _ = env.reset()
obs_opp = env.obs_agent_two()

done = False
trunc = False
step = 0
while not (done or trunc):
    step += 1
    env.render()

    a1_discr = agent_player.act(obs)
    a1 = env.discrete_to_continous_action(a1_discr)
    a2 = agent_opp_strong.act(obs_opp)

    obs, r, done, trunc, _ = env.step(np.hstack([a1, a2]))
    obs_buffer.append(obs)
    reward_buffer.append(r)

    obs_opp = env.obs_agent_two()

    if done or trunc:
        print(f"Episode done in {step} steps")
        break

Episode done in 70 steps


In [9]:
env.close()